# <center> Simulator </center>

This code is used to create synthetic twitter datasets according to the model. We create a user graph and choose an activity pair $(\lambda,\mu)$ for each user . From there we can generate events of tweeting/retweeting where each user $i$ tweets with rate $\lambda_i$ and retweets from his newsfeed with rate $\mu_i$. The output consists of two `.txt` files, one being the adjacency list of the user graph and the other the list of tweets.

In [7]:
import numpy as np
import random as random
from operator import itemgetter

## 2. Timestamps creation

This section is to generate instants of activity within the network. The goal is to ouput a list with each entry of the form `userid timestamp event_type`, where
- `userid` is the unique id $\in \{1, \ldots, N\}$ of the (re)tweeting user
- `timestamp` is the instant of occurence (seconds since the beginning)
- `event_type` is a string that indicates if the event is an original post ('post') or a repost ('repost').

There are 3 functions:
- `exponential_timestamps` creates events with exponential inter-arrival times, i.e. the activity of any user follows a Poisson process
- `hyperexp_timestamps` creates events with hyper-exponential inter-arrival times
- `constant_timestamps` creates events with constant inter-arrival times.

For each one, a maximal number of events must be precised to control the size of the output list. Events are generated user by user, first posts then reposts.

### 2.1 Exponential timestamps
Waiting time between two posts (resp. reposts) from user $i$ is distributed with the density $\lambda_i e^{-\lambda_i x}$ (resp. $\mu_i e^{-\mu_i x}$).

In [8]:
def exponential_timestamps(lambdas, mus, nb_events):
    
    # init
    N = len(lambdas)
    events = list()
    
    # generate user by user
    for j in range(N):
        lambd, mu = lambdas[j], mus[j]
        
        # first posts
        if lambd > 0:
            time = 0
            for n in range(nb_events):
                dice = random.expovariate(lambdas[j])
                time += dice
                events.append((j, time, 'post'))
            
        # then reposts
        if mu > 0:
            time = 0
            for n in range(nb_events):
                dice = random.expovariate(mus[j])
                time += dice
                events.append((j, time, 'repost'))
        
    # end
    return sorted(events, key=itemgetter(1))[:nb_events]

### 2.2 Hyper-exponential timestamps
Waiting time before the next post from user $i$ is distributed as follows:
- with proba $p_i$ it is exponential of parameter $\lambda_i^{(1)}$
- with proba $1 - p_i$ it is exponential of parameter $\lambda_i^{(2)}$

Note that if we set $p_i=10/11$, $\lambda_i^{(1)} = 10 \lambda_i$ and $\lambda_i^{(2)} = 0.1 \lambda_i$ then the inter-arrival times have the same mean that if we used exponential distribution of parameter $\lambda_i$.

Behavior for reposts is similar, with $q$ instead of $p$.

<b> Warning! </b> If $\lambda_i^{(1)} > 0$ we require $\lambda_i^{(2)} > 0$ and reversely. Same for $\mu$.

In [10]:
def hyperexp_timestamps(lambdas1, lambdas2, mus1, mus2, p, q, nb_events):
    
    # init
    N = len(lambdas1)
    events = list()
    
    # generate user by user
    for j in range(N):
        p, q, lambd1, lambd2, mu1, mu2 = p[j], q[j], lambdas1[j], lambdas2[j], mus1[j], mus2[j]
        
        # first posts
        if lambd1 > 0:
            time = 0
            for n in range(nb_events):
                dice = random.random()
                if dice < p:
                    time += random.expovariate(lambd1)
                else:
                    time += random.expovariate(lambd2)
                events.append((n, time, 'post'))
            
        # then reposts
        if mu1 > 0:
            time = 0
            for n in range(nb_events):
                dice = random.random()
                if dice < q:
                    time += random.expovariate(mu1)
                else:
                    time += random.expovariate(mu2)
                events.append((n, time, 'repost'))
        
    # end
    return sorted(events, key=itemgetter(1))[:nb_events]

### 2.3 Constant timestamps
Here we just have to choose an inter-arrival time for each user and it will always be the same. `inter_post` is the list of inter-posting times and `inter_repost` the list of inter-reposting times. Setting one of those to zero makes the user never take the corresponding action (post or repost).

In [12]:
def constant_timestamps(inter_post, inter_repost, nb_events):
    
    # init
    N = len(inter_post)
    events = list()
    
    # generate user by user
    for j in range(N):
        t,s = inter_post[j], inter_repost[j]
        
        # first posts
        if t > 0:
            time = 0
            for n in range(nb_events):
                time += t
                events.append((j, time, 'post'))
            
        # then reposts
        if s > 0:
            time = 0
            for n in range(nb_events):
                time += s
                events.append((j, time, 'repost'))
        
    # end
    return sorted(events, key=itemgetter(1))[:nb_events]

## 3. Final list of events

Here is the function to simulate the evolution of a social platform according to timestamps of activities, as the one that can be generated with the above functions. Parameters:
- `N`: the number of users
- `K`, `M`: walls and newsfeeds sizes
- `timestamps`: list of event of the form as those generated by the previous functions
- `followers`: dictionnary of following relationships, as the one created in the first section (`Followers[i]` is a list containing ids of the followers of user $i$)
- `selection_policy`: string to choose among 'random', 'newest' and 'most_popular'
- `eviction_policy`: string to choose among 'random', 'oldest' and 'least_popular'

Output: a list where the $i^{th}$ entry corresponds to the $i^{th}$ event occurring on the network. Each event is described as a tuple `twid timestamp userid rtid`, with
- `twid` is the unique id of the tweet, $\in \{1, \ldots, nb\_events\}$
- `timestamp` is the instant of occurence (seconds since the beginning)
- `userid` is the unique id $\in \{1, \ldots, N\}$ of the (re)tweeting user
- `rtid` is the id of the original tweet in case of retweet, else is set to -1

If we come accross an impossible event (like a repost from user $i$ but user $i$'s newsfeed is empty) it is skipped. The simulation starts with the first possible event. We keep track of the number of reposts for each content, because it is useful in popular policies.

In [36]:
def simulation(N, M, timestamps, followers, selection_policy='random', eviction_policy='random'):
    
    # initialization
    news = {i:list() for i in range(N)}
    post_id = 1 # id of the next original post
    time = 0 # time counter
    n_iter = len(timestamps) # number of iterations to do
    out_list = list() # will contain the events
    
    # iterations
    for n in range(n_iter):
     
        # get next event
        event_user, event_time, event_type = timestamps[n]
        
        # if repost
        if event_type == 'repost':
            
            if len(news[event_user]) == 0: # skip the event if the user has empty newsfeed
                continue
            else:
                # choose what to repost according to selection policy
                if selection_policy == 'random':
                    new_post = random.choice(news[event_user])
                elif selection_policy == 'newest':
                    new_post = news[event_user][0]
                elif selection_policy == 'most_popular':
                    new_post = max(news[event_user], key=itemgetter(2))
                new_post[2] += 1 # increment number of reposts of the reposted content
            
        # otherwise create new post
        else: 
            new_post = [post_id, event_user, 0]
        
        # update newsfeeds for followers of event user
        for j in range(N):
            if j in followers[event_user]:
                
                # remove post if newsfeed full
                if len(news[j]) == M:
                    if eviction_policy == 'random':
                        deleted_post = random.choice(news[j])
                    elif eviction_policy == 'oldest':
                        deleted_post = news[j][-1]
                    elif eviction_policy == 'least_popular':
                        deleted_post = min(news[j], key=itemgetter(2))
                    news[j].remove(deleted_post)
                
                # insert new post
                news[j].insert(0, new_post)
        
        # add event to list
        uid, ts = event_user, time
        if event_type == 'repost':
            rtid = new_post[0]
        else:
            rtid = -1
        out_list.append((post_id, uid, ts, rtid))
        
        # finally update time and next id
        time = event_time
        post_id += 1
        
    # end return the list of events
    return out_list

## 4. Example

Choose the number of users $N$, the number of events `nb_events` and the activity rates. The latter are in the form of two lists of length $N$: `Lambda` and `Mu` where `Lambda[i]` is the posting rate of user $i$ and `Mu[i]` is her reposting rate.

In [15]:
N = 30
M = 5 # newsfeed size
n_events = 1000
Lambda = np.random.random(N)
Mu = np.random.random(N)

Choose out path where the results will be written.

In [18]:
out_path = "./"

### 4.1. User graph creation

We represent the user graph with a dictionary `Followers` where `Followers[i]` is the set of leaders of user $i$.

In [19]:
# example: graph Erdös-Rényi of parameter w
w = 0.1
Followers = {i:set() for i in range(N)}
for i in range(N):
    for j in range(N):
        if j != i and np.random.random() < w:
            Followers[i].add(j)
print("Number of edges: ", sum([len(Followers[i]) for i in range(N)]))

Number of edges:  106


Write adjacency list on file `out_folder/adjList.txt`.

In [21]:
graph_out = open(out_path + "adjList.txt", "w")
for i in Followers:
    for j in Followers[i]:
        graph_out.write("{} {}\n".format(i,j))
graph_out.close()

### 4.2. Timestamps creation

In [23]:
timestamps = exponential_timestamps(Lambda, Mu, n_events)

Look at the first timestamps.

In [39]:
timestamps[:10]

[(14, 0.031462899399452335, 'repost'),
 (16, 0.056024379106134885, 'post'),
 (23, 0.06419597399917455, 'post'),
 (6, 0.08430047817010024, 'post'),
 (6, 0.10410479471283757, 'repost'),
 (17, 0.2102723355095577, 'repost'),
 (24, 0.22669159946074965, 'post'),
 (8, 0.26083149150247326, 'repost'),
 (6, 0.28329606764354565, 'post'),
 (0, 0.2837111826740648, 'repost')]

### 4.3. Events list creation

In [37]:
Events = simulation(N, M, timestamps, Followers, selection_policy='random', eviction_policy='random')

Look at the first events.

In [38]:
Events[:10]

[(1, 16, 0, -1),
 (2, 23, 0.056024379106134885, -1),
 (3, 6, 0.06419597399917455, -1),
 (4, 24, 0.08430047817010024, -1),
 (5, 8, 0.22669159946074965, 2),
 (6, 6, 0.26083149150247326, -1),
 (7, 17, 0.28329606764354565, -1),
 (8, 19, 0.34406195862671757, -1),
 (9, 18, 0.381316576276887, 8),
 (10, 21, 0.49126426737355666, -1)]

Write events list to `out_path/trace.txt`. Each line is an entry of the list.

In [8]:
out = open(out_path + "trace.txt", "w")
for e in Events:
    out.write("{} {} {} {}\n".format(e[0], e[1], e[2], e[3]))
out.close()